In [ ]:
import os
import re
import json
from bs4 import BeautifulSoup


pasta_paginas = "wikipediapaginas"

pasta_saida = "infobox_json"
os.makedirs(pasta_saida, exist_ok=True)

def sanitizar_nome(nome, limite=100):
    nome = re.sub(r'[\\/*?:"<>|]', "_", nome)
    return nome[:limite].strip()

for arquivo in os.listdir(pasta_paginas):
    if arquivo.endswith(".html"):
        caminho_arquivo = os.path.join(pasta_paginas, arquivo)

        with open(caminho_arquivo, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")

        infobox = soup.find("table", class_="infobox")
        dados_infobox = {}

        if infobox:
            titulo_infobox = infobox.find("th")
            if titulo_infobox:
                titulo = titulo_infobox.get_text(strip=True)
            else:
                titulo = arquivo.replace(".html", "")

            titulo_arquivo_sanitizado = sanitizar_nome(titulo)

            linhas = infobox.find_all("tr")
            for linha in linhas:
                tds = linha.find_all("td")
                ths = linha.find_all("th")

                if len(tds) == 1 and ths:
                    chave = ths[0].get_text(strip=True)
                    valor = tds[0].get_text(separator=", ", strip=True)
                    dados_infobox[chave] = valor

                elif len(tds) == 2:
                    chave = tds[0].get_text(strip=True)
                    td_valor = tds[1]

                    itens_lista = td_valor.find_all("li")
                    if itens_lista:
                        valor = [item.get_text(strip=True) for item in itens_lista]
                    else:
                        valor = td_valor.get_text(separator=", ", strip=True)

                    dados_infobox[chave] = valor


            if dados_infobox:
                caminho_json = os.path.join(pasta_saida, f"{titulo_arquivo_sanitizado}.json")

                with open(caminho_json, "w", encoding="utf-8") as json_file:
                    json.dump(dados_infobox, json_file, ensure_ascii=False, indent=2)


